# 유방암 데이터셋 기반 SVM 분류

In [2]:
import pandas as pd
# 데이터 불러오기
url = 'https://raw.githubusercontent.com/ykkim77/bigdata_analysis_basic_11th/main/cleaned_breast_cancer_data.csv'
cancer = pd.read_csv(url)

cancer.head(5) # 데이터 예시 확인

,id_number,clump_thickness,uniformity_of_cell_size,uniformity_of_cell_shape,marginal_adhesion,single_epithelial_cell_size,bare_nuclei,bland_chromatin,normal_nucleoli,mitoses,target
0,1000025,5,1,1,1,2,1.0,3,1,1,0
1,1002945,5,4,4,5,7,10.0,3,2,1,0
2,1015425,3,1,1,1,2,2.0,3,1,1,0
3,1016277,6,8,8,1,3,4.0,3,7,1,0
4,1017023,4,1,1,3,2,1.0,3,1,1,0


In [3]:
# 독립변수(X)와 종속변수(y) 분리
X = cancer.drop(columns=['id_number', 'target'])  # id는 분석에 불필요하므로 제외
y = cancer['target']

In [4]:
from sklearn.model_selection import train_test_split

# 학습/테스트 데이터 분할 (계층적 샘플링 stratify로 클래스 비율 유지)
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    stratify=y,
    random_state=1
) 

In [ ]:
from sklearn.preprocessing import MinMaxScaler 
scaler=MinMaxScaler()     # 특성값을 [0, 1] 사이로 정규화 

In [6]:
from sklearn.svm import SVC  # SVM(Support Vector Machine) 분류기 클래스 불러오기

# 기본 SVM 모델 (커널 기본값: RBF 커널 사용)
RBF_model = SVC(random_state=1)

# 선형 커널 SVM 모델 정의 (고차원으로 매핑 없이 직선 또는 초평면으로 분리)
linear_model = SVC(kernel="linear", random_state=1)

# 다항식 커널 SVM 모델 정의 (비선형 경계 표현 가능, degree 파라미터로 차수 조절)
poly_model = SVC(kernel="poly", random_state=1)

In [ ]:
from sklearn.pipeline import Pipeline
pipe=Pipeline([('scaler',scaler), ('base_model',poly_model)])

In [ ]:
from sklearn.model_selection import KFold,GridSearchCV 
kfold=KFold(n_splits=3, shuffle=True, random_state=1)

param_grid={'base_model__C':[0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000],
            'base_model__gamma':[0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000],
            'base_model__degree':[1,2,3,4,5]}

# 데이터를 3개의 폴드(fold)로 나눠 학습/검증을 반복, n_jobs=-1: 가능한 모든 CPU 코어를 사용해 병렬처리를 수행
grid_model=GridSearchCV(estimator=pipe, param_grid=param_grid, cv=kfold, n_jobs=-1).fit(X_train, y_train) 

In [ ]:
print('교차검증 점수 : ', grid_model.best_score_)    # accuracy 기준의 교처검증 score
print('최적의 하이퍼 파라메터 조합 :', grid_model.best_params_) 

교차검증 점수 :  0.9765623208347667
최적의 하이퍼 파라메터 조합 : {'base_model__C': 0.0001, 'base_model__degree': 2, 'base_model__gamma': 100}
